# Random dataset generator

In [1]:
from scipy import sparse

def gen_rand_sparse_mat(m, n, density):
    M = sparse.rand(m, n, density=density)
    M.data[:] = 1
    return M

X = gen_rand_sparse_mat(10, 10, 0.1)
print X.toarray()

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  1.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


# Settings

In [18]:

ITEMS = range(1000)
TAGS = range(100)
ITEM_COUNT = len(ITEMS)
TAG_COUNT = len(TAGS)
DENSITY = 0.15

# X: tag matrix
# T: maximun number of questions
# GAMMA: discount factor
X = gen_rand_sparse_mat(ITEM_COUNT, TAG_COUNT, DENSITY)
T = 5
GAMMA = 0.75

print X.toarray()

[[ 0.  1.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


# Question selection al.: Greedy

In [42]:
import numpy as np
from scipy import sparse

def questionSellectionAlgo_greedy(X, w, qs):
    X_qs = X.toarray()[:, qs]
    E = np.dot(w, X_qs) / w.sum()
    q = np.argmin(np.fabs(E - 0.5))
        
    return qs[q]
    
print questionSellectionAlgo_greedy(X, np.ones(ITEM_COUNT), range(TAG_COUNT))

1


# Question selection al.: IHS_Exhaustion

In [ ]:

def questionSellectionAlgo_ihs_exhaustion(X, GAMMA, w, qs, k):
    ls = (((np.array(range(2**k))[:,None] & (1 << np.arange(k)))) > 0).astype(int)
    qks = [qk for qk in itertools.combinations(qs, k)]
    
    min_ls = []
    for qk in qks:   
        X_qk = X.toarray()[:, [qs[q] for q in qk]].astype(int)
        
        ls_ds = []
        for l in ls:
            dcs = [np.bitwise_xor(l, X_qki).sum() for X_qki in X_qk]
            d = (w*[1 - GAMMA**dc for dc in dcs]).sum()
            ls_ds.append(d)
        min_ls.append(np.min(ls_ds))
    
    Q = np.argmax(min_ls)
    
    return [qs[q] for q in qks[Q]]

# may take about 2 min for k=2 on 4G RAM
#print questionSellectionAlgo_ihs_exhaustion(X, GAMMA, np.ones(ITEM_COUNT), range(TAG_COUNT), 2)

# Question selection al.: IHS_SA (Simulating Annealing)

In [41]:
import random

def jump_prob(e, enew, T):
    if enew < e: 
        return 1
    else:
        return np.exp((e-enew)/T)

def compute_min_d(X_qk, GAMMA, w, ls):
    ls_ds = []
    for l in ls:
        xor = (l+X_qk)
        xor[xor!=1] = 0
        dcs = [x.sum() for x in xor]
        d = (w*[1 - GAMMA**dc for dc in dcs]).sum()
        ls_ds.append(d)
    return np.min(ls_ds)


def questionSellectionAlgo_ihs_sa(X, GAMMA, w, qs, k, I, T):
    if len(qs) <= k:
        return qs
    ls = (((np.array(range(2**k))[:,None] & (1 << np.arange(k)))) > 0).astype(int)
    qk = np.random.choice(qs, k, replace=False)
    X_qk = X.toarray()[:, qk]
    min_d = compute_min_d(X_qk, GAMMA, w, ls)
    t = T
    
    qk_best = qk
    for i in range(I):
        qk_n = np.array(qk)
        qk_n[np.random.choice(k)] = np.random.choice(np.setdiff1d(qs, qk))
        X_qk_n = X.toarray()[:, qk_n]
        min_d_n = compute_min_d(X_qk_n, GAMMA, w, ls)
        
        t = t*0.95
        #print jump_prob(min_d, min_d_n, t)
        if jump_prob(min_d, min_d_n, t) > np.random.rand(1)[0]:
            qk = qk_n
        if min_d_n > min_d:
            min_d = min_d_n
            qk_best = qk_n
    return qk_best

#for i in range(300):
print questionSellectionAlgo_ihs_sa(X, GAMMA, np.ones(ITEM_COUNT), range(TAG_COUNT), 2, 1000, 50)

[1 6]


# Interactive recommandation: greedy & ihs_sa

In [10]:

def interactiveRecommandation(X, T, GAMMA):
    r_qs = np.array(range(TAG_COUNT)) # remain questions: had not asked
    w = np.ones(ITEM_COUNT)
    
    for t in range(T):
        q = questionSellectionAlgo_greedy(X, w, r_qs)
        #q = questionSellectionAlgo_ihs_sa(X, GAMMA, w, r_qs, 2, 1000, 100)[0]
        r_qs = r_qs[r_qs != q]
        
        X_q = X.toarray()[:, q]
        
        ans = raw_input('has ' + str(TAGS[q]) + '? (0/1): ')
        g = np.ones(ITEM_COUNT)
        if ans == '1':
            g = X_q
        else:
            g = (1 - X_q)
        g[g < 1] = GAMMA
        w = w*g
        
        print 'W: ', w
    return w

interactiveRecommandation(X, T, GAMMA)

has 92? (0/1): 1
W:  [ 0.75  0.75  1.    0.75  0.75  0.75  0.75  1.    1.    0.75  1.    0.75
  0.75  0.75  0.75  0.75  1.    1.    0.75  0.75  1.    1.    0.75  0.75
  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75
  0.75  0.75  0.75  0.75  1.    0.75  1.    0.75  0.75  0.75  0.75  0.75
  0.75  1.    0.75  0.75  0.75  0.75  1.    0.75  1.    0.75  0.75  0.75
  0.75  0.75  1.    0.75  0.75  0.75  0.75  0.75  1.    0.75  0.75  0.75
  0.75  0.75  0.75  0.75  0.75  1.    1.    0.75  0.75  0.75  0.75  0.75
  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  1.    0.75
  0.75  0.75  0.75  0.75  1.    0.75  0.75  0.75  1.    0.75  0.75  0.75
  0.75  0.75  0.75  0.75  0.75  1.    0.75  0.75  0.75  0.75  0.75  1.
  0.75  1.    0.75  0.75  0.75  0.75  1.    0.75  0.75  1.    0.75  0.75
  0.75  1.    0.75  0.75  0.75  0.75  0.75  0.75  0.75  1.    0.75  1.
  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  1.    0.75  0.75
  1.    0.75  0.75  0.75  1.    0.

array([ 0.31640625,  0.23730469,  0.31640625,  0.31640625,  0.31640625,
        0.23730469,  0.23730469,  0.31640625,  0.421875  ,  0.31640625,
        0.31640625,  0.5625    ,  0.31640625,  0.23730469,  0.23730469,
        0.23730469,  0.31640625,  0.31640625,  0.23730469,  0.31640625,
        0.31640625,  0.421875  ,  0.23730469,  0.31640625,  0.23730469,
        0.421875  ,  0.23730469,  0.31640625,  0.23730469,  0.31640625,
        0.23730469,  0.5625    ,  0.23730469,  0.31640625,  0.31640625,
        0.23730469,  0.23730469,  0.31640625,  0.31640625,  0.23730469,
        0.421875  ,  0.31640625,  0.421875  ,  0.31640625,  0.23730469,
        0.23730469,  0.5625    ,  0.23730469,  0.31640625,  0.421875  ,
        0.23730469,  0.31640625,  0.23730469,  0.23730469,  0.421875  ,
        0.23730469,  0.421875  ,  0.23730469,  0.23730469,  0.31640625,
        0.31640625,  0.421875  ,  0.421875  ,  0.31640625,  0.23730469,
        0.31640625,  0.31640625,  0.31640625,  0.421875  ,  0.42

# Interactive Recommandation: ihs_shs for k=2

In [11]:

def interactiveRecommandation_shs(X, C, GAMMA, k, I, T):
    r_qs = np.array(range(TAG_COUNT)) # remain questions: had not asked
    w = np.ones(ITEM_COUNT)
    
    t = 0
    while t != C:
        qk = questionSellectionAlgo_ihs_sa(X, GAMMA, w, r_qs, k, I, T)
        print qk
        for q in qk: # ask k questions, no interaction with user
            if t == C:
                break
            r_qs = r_qs[r_qs != q]
        
            X_q = X.toarray()[:, q]
        
            ans = raw_input('has ' + str(TAGS[q]) + '? (0/1): ')
            g = np.ones(ITEM_COUNT)
            if ans == '1':
                g = X_q
            else:
                g = (1 - X_q)
            g[g < 1] = GAMMA
            w = w*g
        
            t = t + 1
            print 'W: ', w
    return w

interactiveRecommandation_shs(X, T, GAMMA, 2, 1000, 100)

[ 8 92]
has 8? (0/1): 1
W:  [ 0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  1.
  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  1.
  0.75  1.    0.75  0.75  0.75  0.75  0.75  1.    0.75  1.    1.    0.75
  0.75  0.75  0.75  0.75  1.    0.75  0.75  0.75  0.75  0.75  0.75  0.75
  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  1.    0.75  0.75  0.75
  1.    0.75  0.75  0.75  0.75  0.75  1.    1.    0.75  1.    0.75  0.75
  0.75  0.75  1.    1.    0.75  0.75  0.75  0.75  0.75  0.75  1.    0.75
  0.75  0.75  1.    0.75  1.    0.75  1.    0.75  0.75  0.75  0.75  1.
  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75
  0.75  0.75  0.75  0.75  0.75  1.    0.75  0.75  0.75  0.75  1.    0.75
  0.75  1.    0.75  1.    0.75  1.    0.75  0.75  0.75  1.    1.    0.75
  0.75  0.75  1.    0.75  0.75  0.75  0.75  0.75  0.75  1.    0.75  0.75
  0.75  0.75  0.75  0.75  0.75  1.    0.75  0.75  0.75  0.75  0.75  1.
  0.75  1.    0.75  1.    0.75 

array([ 0.23730469,  0.31640625,  0.421875  ,  0.421875  ,  0.31640625,
        0.31640625,  0.31640625,  0.5625    ,  0.5625    ,  0.23730469,
        0.421875  ,  0.421875  ,  0.421875  ,  0.421875  ,  0.31640625,
        0.31640625,  0.5625    ,  0.421875  ,  0.31640625,  0.421875  ,
        0.421875  ,  0.5625    ,  0.31640625,  0.421875  ,  0.31640625,
        0.31640625,  0.31640625,  0.31640625,  0.31640625,  0.31640625,
        0.421875  ,  0.421875  ,  0.31640625,  0.421875  ,  0.421875  ,
        0.31640625,  0.421875  ,  0.23730469,  0.421875  ,  0.421875  ,
        0.5625    ,  0.31640625,  0.31640625,  0.23730469,  0.421875  ,
        0.31640625,  0.31640625,  0.31640625,  0.31640625,  0.421875  ,
        0.31640625,  0.5625    ,  0.31640625,  0.31640625,  0.31640625,
        0.31640625,  0.5625    ,  0.421875  ,  0.31640625,  0.31640625,
        0.421875  ,  0.31640625,  0.75      ,  0.23730469,  0.31640625,
        0.23730469,  0.421875  ,  0.421875  ,  0.5625    ,  0.56

# Experiment algorithms: return question count

In [40]:
def is_user_error(p):
    return np.random.choice(2, p=[1-p, p]) # error when 1 generated


def ir_greedy(X, target, p, QC, GAMMA):
    r_qs = np.array(range(TAG_COUNT)) # remain questions: had not asked
    w = np.ones(ITEM_COUNT)
    
    for t in range(QC):
        q = questionSellectionAlgo_greedy(X, w, r_qs)
        r_qs = r_qs[r_qs != q]
        
        err = is_user_error(p)
        true_ans = X.toarray()[target][q]
        ans = true_ans if (err == 0) else 1-true_ans
        
        g = np.ones(ITEM_COUNT)
        X_q = X.toarray()[:, q]
        if ans == 1:
            g = X_q
        else:
            g = (1 - X_q)
        g[g < 1] = GAMMA
        w = w*g
        
        if np.argmax(w) == target:
            return t+1
    return QC

def ir_ihs(X, target, p, QC, GAMMA):
    r_qs = np.array(range(TAG_COUNT)) # remain questions: had not asked
    w = np.ones(ITEM_COUNT)
    
    for t in range(QC):
        q = questionSellectionAlgo_ihs_sa(X, GAMMA, w, r_qs, 2, 100, 50)[0] # issue: iteration count & efficency
        r_qs = r_qs[r_qs != q]
        
        err = is_user_error(p)
        true_ans = X.toarray()[target][q]
        ans = true_ans if (err == 0) else 1-true_ans
        
        g = np.ones(ITEM_COUNT)
        X_q = X.toarray()[:, q]
        if ans == 1:
            g = X_q
        else:
            g = (1 - X_q)
        g[g < 1] = GAMMA
        w = w*g
        
        if np.argmax(w) == target:
            return t+1
    return QC

def ir_shs(X, target, p, QC, GAMMA):
    r_qs = np.array(range(TAG_COUNT)) # remain questions: had not asked
    w = np.ones(ITEM_COUNT)
    
    t = 0
    while t != QC:
        qk = questionSellectionAlgo_ihs_sa(X, GAMMA, w, r_qs, 2, 100, 50) # issue: iteration count & efficency
        for q in qk: # ask k questions, no interaction with user
            if t == QC:
                return QC
            r_qs = r_qs[r_qs != q]
            
            err = is_user_error(p)
            true_ans = X.toarray()[target][q]
            ans = true_ans if (err == 0) else 1-true_ans
            
            g = np.ones(ITEM_COUNT)
            X_q = X.toarray()[:, q]
            if ans == 1:
                g = X_q
            else:
                g = (1 - X_q)
            g[g < 1] = GAMMA
            w = w*g
            
            if np.argmax(w) == target:
                return t+1
            t = t + 1
    return QC



# Experiments: user error & density of matrix

In [43]:
PS = np.arange(0., 0.12, 0.02, dtype=np.float) # prob. of user error: [ 0.    0.02  0.04  0.06  0.08  0.1 ]
DENS = np.arange(0.05, 0.55, 0.05, dtype=np.float) # densities: [ 0.05  0.1   0.15  0.2   0.25  0.3   0.35  0.4   0.45  0.5 ]
TARGET_ITEM_COUNT = 300

ITEMS = range(1000)
TAGS = range(100)
ITEM_COUNT = len(ITEMS)
TAG_COUNT = len(TAGS)

QC = TAG_COUNT
GAMMA = 0.

for p in PS:
    for den in DENS:
        X = gen_rand_sparse_mat(ITEM_COUNT, TAG_COUNT, den)
        targets = np.random.choice(ITEMS, TARGET_ITEM_COUNT, replace=False) # select 300 target item to answer questions
        
        tt_qs = np.array([0, 0, 0]) # [tt_greedy, tt_ihs, tt_shs]
        for tar in targets:
            tt_qs += [ir_greedy(X, tar, p, QC, GAMMA), ir_ihs(X, tar, p, QC, GAMMA), ir_shs(X, tar, p, QC, GAMMA)]
        avg_qs = tt_qs/TARGET_ITEM_COUNT
        
        print avg_qs

KeyboardInterrupt: 

In [ ]:
res_g075 = [[[32, 36, 38],[18, 22, 23],[14, 16, 17],[12, 14, 14],[11, 12, 12],[10, 11, 11],[10, 10, 10],[ 9, 10, 10],[9, 9, 9],[9, 9, 9]], 
           [[34, 39, 37],[20, 24, 25],[15, 18, 19],[13, 15, 15],[11, 13, 13],[11, 12, 11],[11, 11, 11],[10, 11, 11],[10, 10, 10],[10, 10, 10]], 
           [[39, 42, 44],[21, 27, 26],[16, 21, 20],[14, 16, 17],[12, 14, 14],[11, 13, 13],[11, 12, 12],[11, 11, 11],[11, 11, 11],[11, 11, 11]], 
           [[45, 49, 50],[23, 30, 29],[18, 22, 23],[15, 17, 17],[14, 16, 16],[12, 15, 14],[12, 13, 13],[12, 12, 13],[12, 12, 12],[12, 12, 12]], 
           [[49, 56, 56],[26, 33, 33],[19, 24, 24],[17, 21, 20],[15, 18, 17],[15, 15, 16],[13, 14, 14],[14, 14, 13],[13, 14, 14],[14, 14, 13]], 
           [[55, 63, 57],[32, 39, 39],[22, 28, 28],[19, 22, 21],[17, 19, 19],[15, 16, 17],[15, 17, 16],[14, 15, 15],[15, 14, 15],[15, 15, 15]]]
print res_g075